In [41]:
import numpy as np
import pandas as pd
import torch
from allennlp.predictors.predictor import Predictor

In [53]:
predictor_small = Predictor.from_path("ser_dir_original_model/model.tar.gz")
predictor_large = Predictor.from_path("ser_dir_large_model/model.tar.gz")

Преобразуем данные из test.csv, парсим по колонкам

In [71]:
test_df = pd.read_csv('test.csv')[['Event','Xintent','Xemotion','Otheremotion']]
def str2list(x):
    return [y.strip() for y in x[2:-2].replace('"','').replace("'",'').split(',')]
for col in ['Xintent','Xemotion','Otheremotion']:
    test_df[col] = test_df[col].apply(str2list)
    
test_df[1600:1700]

,Event,Xintent,Xemotion,Otheremotion
1600,PersonX puts paid to ___,"[to finalize, to complete]","[final, completed]",[none]
1601,PersonX fights a losing battle,[none],"[discouraged, beaten, sad]",[sympathetic]
1602,PersonX fights a losing battle,[doesnt want to give up],[proud and persistant],[none]
1603,PersonX gathers dust,[none],[bad for not having anything constructive to do],[none]
1604,PersonX gets a handle on ___,[to learn more about a subject.],[informed.],[none]
1605,PersonX gets a handle on ___,[none],[satisfied],[none]
1606,PersonX stems the tide,[stop things from happening],[heroic],[safe]
1607,PersonX stems the tide,[slow or stop something],[nan],[none]
1608,PersonX stems the tide,"[to shut down something, to stop questions]",[relieved],[frustrated]
1609,PersonX sifts out ___,[separate things.],[satisfied.],[none]


Группируем по фразам отдельно намерения, эмоции субъекта и эмоции окружающих

In [77]:
phrase2oemotion = {}
phrase2xemotion = {}
phrase2xintent = {}
res_xi = []
res_xe = []
res_oe = []

phrase_set = set([x for x in test_df.Event])

for phrase in phrase_set:
    for x in test_df[test_df.Event == phrase].Xintent:
        res_xi += x
    phrase2xintent[phrase] = set(res_xi)
    res_xi.clear()
    for x in test_df[test_df.Event == phrase].Xemotion:
        res_xe += x
    phrase2xemotion[phrase] = set(res_xe)
    res_xe.clear()
    for x in test_df[test_df.Event == phrase].Otheremotion:
        res_oe += x
    phrase2oemotion[phrase] = set(res_oe)
    res_oe.clear()

In [72]:
p = "PersonX takes a nap"

In [73]:
phrase2xintent[p]

{'is tired',
 'to help a headache go away.',
 'to not be tired',
 'to rest.',
 'to sleep',
 'to sleep.',
 'to take a break'}

In [76]:
result = predictor_large.predict(source=p)

oreact = set([" ".join(react) for react in result["oreact_top_k_predicted_tokens"]])    
intents = set([" ".join(react) for react in result["xintent_top_k_predicted_tokens"]])
xreact = set([" ".join(react) for react in result["xreact_top_k_predicted_tokens"]])

intents

{'get rest',
 'get some rest',
 'get to work',
 'have fun',
 'relax',
 'rest',
 'rest .',
 'rested',
 'sleep',
 'take rest'}

Считаем recall следующим образом: из всех вариантов, выданных моделью для данной фразы, оставляем те, что есть в контрольном множестве фразы, сформированном ранее, и делим их количество на размер контрольного множества. Затем делим полученную сумму на количество фраз. Проделываем для каждого из выходов (intent, xreact и oreact)

In [83]:
res_oe = 0
res_xe = 0
res_xi = 0

for phrase in phrase_set:
    result = predictor_small.predict(source=phrase)

    oreact = set([" ".join(react) for react in result["oreact_top_k_predicted_tokens"]])    
    intents = set([" ".join(react) for react in result["xintent_top_k_predicted_tokens"]])
    xreact = set([" ".join(react) for react in result["xreact_top_k_predicted_tokens"]])

    res_oe += len(phrase2oemotion[phrase].intersection(oreact)) / len(phrase2oemotion[phrase])
    res_xe += len(phrase2xemotion[phrase].intersection(xreact)) / len(phrase2xemotion[phrase])
    res_xi += len(phrase2xintent[phrase].intersection(intents)) / len(phrase2xintent[phrase])

print("Recall for intents: ", res_xi / len(phrase_set))
print("Recall for x_react: ", res_xe / len(phrase_set))
print("Recall for o_react: ", res_oe / len(phrase_set))

Recall for intents:  0.20332426906132062
Recall for x_react:  0.23903328485001757
Recall for o_react:  0.6408387613566894


In [ ]:
res_oe = 0
res_xe = 0
res_xi = 0

for phrase in phrase_set:
    result = predictor_large.predict(source=phrase)

    oreact = set([" ".join(react) for react in result["oreact_top_k_predicted_tokens"]])    
    intents = set([" ".join(react) for react in result["xintent_top_k_predicted_tokens"]])
    xreact = set([" ".join(react) for react in result["xreact_top_k_predicted_tokens"]])

    res_oe += len(phrase2oemotion[phrase].intersection(oreact)) / len(phrase2oemotion[phrase])
    res_xe += len(phrase2xemotion[phrase].intersection(xreact)) / len(phrase2xemotion[phrase])
    res_xi += len(phrase2xintent[phrase].intersection(intents)) / len(phrase2xintent[phrase])

print("Recall for intents: ", res_xi / len(phrase_set))
print("Recall for x_react: ", res_xe / len(phrase_set))
print("Recall for o_react: ", res_oe / len(phrase_set))

In [ ]:
res_oe = 0
res_xe = 0
res_xi = 0
count = 0

for phrase in phrase_set:
    result = predictor_large.predict(source=phrase)

    oreact = set([" ".join(react) for react in result["oreact_top_k_predicted_tokens"]])    
    intents = set([" ".join(react) for react in result["xintent_top_k_predicted_tokens"]])
    xreact = set([" ".join(react) for react in result["xreact_top_k_predicted_tokens"]])

    res_oe += len(phrase2oemotion[phrase].intersection(oreact)) / len(phrase2oemotion[phrase]) * test_df[test_df.Event == phrase].shape[0]
    res_xe += len(phrase2xemotion[phrase].intersection(xreact)) / len(phrase2xemotion[phrase]) * test_df[test_df.Event == phrase].shape[0]
    res_xi += len(phrase2xintent[phrase].intersection(intents)) / len(phrase2xintent[phrase]) * test_df[test_df.Event == phrase].shape[0]
    
    count += test_df[test_df.Event == phrase].shape[0]

print("Recall for intents: ", res_xi / count)
print("Recall for x_react: ", res_xe / count)
print("Recall for o_react: ", res_oe / count)